In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import datetime
import urllib.request
import pandas as pd
from selenium.webdriver.support.select import Select
import numpy as np

In [2]:
# Price Type Selection
pricetype = "W" # W = Wholesale, R = Retail

# Date Generator
start_date = "12/01/2018"
end_date = "12/01/2019"

start = datetime.datetime.strptime(start_date, "%m/%d/%Y")
end = datetime.datetime.strptime(end_date, "%m/%d/%Y")

date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

date_list = []

for date in date_generated:
    date_list.append(date.strftime("%m/%d/%Y"))

In [3]:
urlpage = "http://kalimatimarket.gov.np/daily-price-information"

# Query the website and run the html to the variable "page"
page = urllib.request.urlopen(urlpage)

# Parse the html using Beautiful Soup and Store in variable Soup
soup = BeautifulSoup(page, 'html.parser')

# Find the DatePicker Input
date = soup.find("input", id="datepicker", attrs={'name': "date"})
viewButton = soup.find("input", attrs={'type': 'button', "name":"view"})

In [4]:
# Run Chromium Browser
driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver')
# Get webpage
driver.get(urlpage)

In [5]:
# Insert the Custom date to INPUT Tag
date = "11/04/2014"
# datepicker = driver.find_element_by_tag_name("input")
datepicker = driver.find_element_by_xpath("//input[contains(@id, 'datepicker')]")
driver.execute_script("arguments[0].value = '"+date+"';", datepicker)

In [6]:
# Select pricetype
# price_type = driver.find_element_by_xpath("//select[contains(@id, 'pricetype')]").click()
select = Select(driver.find_element_by_id("pricetype"))

# Select by Index
# select.select_by_index(1)

# select by value
select.select_by_value(pricetype)

In [7]:
# Click the View option after inserting Date
view_button = driver.find_element_by_xpath("//input[contains(@name, 'view')]")
driver.execute_script("arguments[0].click();",view_button)

In [8]:
# import csv

In [9]:
# csv_file = open("./source/kalimatiprice1.csv", 'w')
# csv_writer = csv.writer(csv_file)
# csv_writer.writerow([f"{datetime.datetime.now():%Y-%m-%d}"])
# csv_writer.writerow(['Product Name', 'Unit', 'Minimum', 'Maximum', 'Average'])

In [10]:
table_rows = driver.find_elements_by_xpath("//div[contains(@id, 'pricelist')]/center/table/tbody/tr/td/table/tbody/tr")
row_array = np.array(table_rows)
data =[]
for each_row in row_array:
    data.append(each_row.text)
# row_array[3].text.split(' ')
data_array = np.array(data[3:])
type(data_array[0].split(' '))
data_splited = []
for i in range(len(data_array)):
    data_splited.append(data_array[i].split(' '))

In [11]:
def correct_data(table_row):
    table_with_name = ''
    table_with_price = []
    first_element_array = []
    if len(data_splited[1]) > 5:
    #     table_with_price = table_row[:-4] + table_row[-4:]
        for i in range(len(table_row) - len(table_row[-4:])):
            table_with_name += table_row[i] + " "
        first_element_array.insert(0, table_with_name)
        table_with_price = first_element_array + table_row[-4:]
    else:
        table_with_price = table_row
    return table_with_price

In [12]:
final_data = []
for i in range(len(data_splited)):
    final_data.append(correct_data(data_splited[i]))


In [14]:
df = pd.DataFrame(final_data, columns = ["Product", "Unit", "Minimum", "Maximum", "Average"])
df.tail()

,Product,Unit,Minimum,Maximum,Average
64,लसुन सुकेको चाइनिज,के.जी.,११५,१२०,११७
65,लसुन सुकेको नेपाली,के.जी.,८०,९०,८७
66,छ्यापी सुकेको,के.जी.,११०,१२०,११७
67,छ्यापी हरियो,के.जी.,९०,१००,९५
68,माछा ताजा,के.जी.,२२०,२३०,२२७


In [15]:
export_csv = df.to_csv ('./source/export_dataframe.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [ ]:
for date in date_list:
    print("./source/kalimati/retail/"+date.replace('/', '-')+".csv")